# Table to LLM
- Will rip HTML from websites
- Raw HTML will be either:
    - Processed to remove most tags using some Python
    - Porcessed directly by the LLM
- Chunk, embed and upsert the final data into a vector database
- Query it with user input

In [26]:
# Setup

import os
import re
import json
import uuid
import yaml
import tiktoken
import requests
import pandas as pd
from io import StringIO
from dotenv import load_dotenv
from bs4 import BeautifulSoup
import markdown
import chromadb
from chromadb.config import Settings

In [27]:
# Vars

load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
together_api_key = os.getenv('TOGETHER_API_KEY')

In [28]:
# Data

data_types = ['txt', 'html', 'json', 'md', 'yaml']
file_df = pd.read_csv(r"..\Data\Other\html_data.csv")
file_df.head(3)

,Index,URL,Num of Tables,Pictures of Tables,Dynamic Dropdown,Note
0,1,https://support.microsoft.com/en-us/office/int...,1,0,No,-
1,2,https://support.microsoft.com/en-us/office/for...,1,0,No,-
2,3,https://support.microsoft.com/en-us/office/vid...,1,0,No,-


In [29]:
# ChromaDB

chroma_client = chromadb.Client(settings=Settings(allow_reset=True))
chroma_client.reset()
collection = chroma_client.create_collection(name="copilot")

## Functions

In [30]:
# Data File Related

def fetch_html(url):
    try:
        response = requests.get(url)
        if response.status_code == 200:
            return response.text
        else:
            print(f"Failed to retrieve HTML. Status code: {response.status_code}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

def get_all_urls():
    urls = file_df['URL'].tolist()
    return urls

In [31]:
# LLM API Functions

def make_openai_call(payload):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {openai_api_key}',
    }
    response = requests.post('https://api.openai.com/v1/chat/completions', json=payload, headers=headers)
    
    result = response.json()

    input_tokens = result['usage']['prompt_tokens']
    output_tokens = result['usage']['completion_tokens']
    result_text = result['choices'][0]['message']['content']

    return input_tokens, output_tokens, result_text
    
def make_together_call(payload):
    headers = {
        'Content-Type': 'application/json',
        'Authorization': f'Bearer {together_api_key}',
    }
    # print(headers)
    response = requests.post('https://api.together.xyz/v1/chat/completions', json=payload, headers=headers)
    # print(response)
    
    result = response.json()
    # print(result)

    result_text = result['choices'][0]['message']['content']
    # print(result_text)

    return result_text

In [32]:
# Chunking Related

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

def chunk_text(text, encoding_name, max_tokens=2000):
    """
    Splits text into chunks with each chunk having a maximum of max_tokens tokens.
    """
    chunks = []
    current_chunk = ""
    words = text.split()
    for word in words:
        # Simulate adding the word to the current chunk and check the token count
        test_chunk = current_chunk + " " + word if current_chunk else word
        if num_tokens_from_string(test_chunk, encoding_name) <= max_tokens:
            current_chunk = test_chunk
        else:
            # Current chunk is full, start a new one
            chunks.append(current_chunk)
            current_chunk = word
    # Add the last chunk if it's not empty
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

In [33]:
# Extract DF from Formats

def extract_tables_from_html(html_content: str) -> pd.DataFrame:
    """Extract tables from HTML content and return them as a list of pandas DataFrames."""
    soup = BeautifulSoup(html_content, 'html.parser')
    html_io = StringIO(html_content)
    return pd.read_html(html_io)[0]
    # return [pd.read_html(str(table))[0] for table in soup.find_all('table')]

def extract_tables_from_json(json_content: str) -> pd.DataFrame:
    """Extract tables from JSON content and return them as a list of pandas DataFrames."""
    data = json.loads(json_content)
    return pd.DataFrame(list(data.values())[0])
    # return [pd.DataFrame(table_data) for table_data in data.values()]

def extract_tables_from_md(md_content: str) -> pd.DataFrame:
    """Convert Markdown content to HTML and extract tables from it."""
    html_content = markdown.markdown(md_content)
    return extract_tables_from_html(html_content)
    # return extract_tables_from_html(html_content)

def extract_tables_from_file(html_text: str, source: str) -> pd.DataFrame:
    """Extract tables from HTML content string based on its extension."""

    if source == "html":
        return extract_tables_from_html(html_text)
    elif source.endswith('.json'):
        return extract_tables_from_json(html_text)
    elif source.endswith('.md'):
        return extract_tables_from_md(html_text)
    else:
        raise ValueError("Unsupported file format")

In [34]:
# Convert DF to Formats

def dataframe_to_text(dataframe: pd.DataFrame) -> str:
    """Convert a pandas DataFrame to a text format."""
    return dataframe.to_string(index=False)

def dataframe_to_html(dataframe: pd.DataFrame) -> str:
    """Convert a pandas DataFrame to HTML format."""
    return dataframe.to_html(index=False)

def dataframe_to_json(dataframe: pd.DataFrame) -> str:
    """Convert a pandas DataFrame to JSON format."""
    return dataframe.to_json(orient='records', indent=4)

def dataframe_to_md(dataframe: pd.DataFrame) -> str:
    """Convert a pandas DataFrame to Markdown format by first converting to HTML."""
    return markdown.markdown(dataframe.to_html(index=False))

def dataframe_to_yaml(dataframe: pd.DataFrame) -> str:
    """Convert a pandas DataFrame to YAML format."""
    return yaml.dump(dataframe.to_dict(orient='records'), default_flow_style=False)

def print_format_from_table(print_format: str, dataframe: pd.DataFrame) -> None:
    """Print or save a pandas DataFrame in the specified format."""
    format_functions = {
        'html': dataframe_to_html,
        'json': dataframe_to_json,
        'md': dataframe_to_md,
        'txt': dataframe_to_text,
        'yaml': dataframe_to_yaml
    }
    
    if print_format not in format_functions:
        raise ValueError("Unsupported save format")

    print_content = format_functions[print_format](dataframe)
    return print_content

In [35]:
# HTML Processing with Python/LLM

def python_strip_most_tags(html_content: str) -> str:
    # Remove most HTML tags and convert the table tag content into whatever format needed
    
    # Parse HTML content
    soup = BeautifulSoup(html_content, 'html.parser')
    
    # Remove all tags except for <table> tags
    for tag in soup.find_all(True):
        if tag.name != 'table':
            tag.unwrap()

    # Get text
    cleaned_html = soup.get_text(separator=' ')
    cleaned_html = re.sub(r'\s+', ' ', cleaned_html).strip()

    return cleaned_html

def llm_strip_most_tags(html_content: str) -> str:
    # Remove most HTML tags and convert the table tag content into whatever format needed

    num_tokens_in_html = num_tokens_from_string(html_content, "cl100k_base")
    print(F"HTML contains {num_tokens_in_html} tokens")

    # Call split_html function to split html_content
    html_parts = chunk_text(html_content, "cl100k_base", 2048)
    
    # Initialize cleaned_html variable to store concatenated results
    cleaned_html = ''
    
    # Loop through each part of the html_content
    for part in html_parts:
        payload = {
            'messages': [
                {
                    'role': 'system',
                    'content': f"You are a helpful assistant that takes as input raw HTML and returns it cleaned with no HTML tags. ONLY returned cleaned result."
                },
                {
                    'role': 'user', 
                    'content': f"{part}"
                }
            ],
            'model': "openchat/openchat-3.5-1210",
            'max_tokens': 2048,
            "temperature": 0.0
        }
        print(payload)
        cleaned_html += make_together_call(payload)
    
    return cleaned_html    

In [74]:
def process_html(raw_html, output_format):
    soup = BeautifulSoup(raw_html, 'html.parser')

    # Find all table elements
    tables = soup.find_all('table')
    # print(f"len of tables: {len(tables)}")
    
    top_level_tables = [table for table in tables if not table.find_parent('table')]
    # print(f"len of tables: {len(top_level_tables)}")

    # Filter out tables that are just wrappers for images or have insufficient content
    tables_with_content = []
    for table in top_level_tables:
        # Count all the elements within each table
        all_elements = table.find_all(True)
        # Count all the img tags within each table
        img_elements = table.find_all('img')
        
        # Count non-empty cells and img wrappers
        non_empty_cells = [cell for cell in table.find_all('td') if cell.get_text(strip=True)]
        img_wrappers = [cell for cell in table.find_all('td') if cell.find('img')]
        
        # Adjust condition to filter out tables that are primarily image wrappers or have insufficient content
        if len(non_empty_cells) > 0 and len(all_elements) - len(img_elements) > len(img_wrappers):
            tables_with_content.append(table)
    # print(f"Filtered top level tables count: {len(tables_with_content)}")

    for i, table in enumerate(tables_with_content):
        # print(f"Table {i}\n{table}\n")
        
        soup_table_df = extract_tables_from_file(str(table), "html")
        # print(soup_table_df)
        
        soup_table_format = print_format_from_table(output_format, soup_table_df)
        # print(soup_table_format)

        payload = {
            'messages': [
                {
                    'role': 'system',
                    'content': f"You are a helpful assistant that takes as input raw dump of a table and summarizes it. Create a title for the table and explain each row in a line of text."
                },
                {
                    'role': 'user', 
                    'content': f"{soup_table_format}"
                }
            ],
            'model': "openchat/openchat-3.5-1210",
            'max_tokens': 2048,
            "temperature": 0.0
        }
        soup_table_summary = make_together_call(payload)
        # print(soup_table_summary)
        
        # Create new elements with BeautifulSoup
        title_element = soup.new_string("\n" + soup_table_summary + "\n\n")
        table.insert_before(title_element)
        table.replace_with("\n")

    # Remove all tags from the soup and keep only text
    text_only = soup.get_text(separator=' ', strip=True)

    return text_only

## Single Table Testing

In [75]:
# Retrieve all URLs, clean and append content, metadata and IDs

all_urls = get_all_urls()
test_url = all_urls[11]
print(f"test_url: {test_url}\n")
html = fetch_html(test_url)
if html:
    cleaned_html = process_html(html, "html")
    print(f"cleaned_html\n{cleaned_html}\n")

test_url: https://support.microsoft.com/en-us/office/calculate-values-in-a-pivottable-11f41417-da80-435c-a5c6-b0185e59da77

Filtered top level tables count: 4

cleaned_html
Calculate values in a PivotTable - Microsoft Support Skip to main content Microsoft Support Support Support Home Microsoft 365 Office Products Microsoft 365 Outlook Microsoft Teams OneDrive OneNote Windows Microsoft Edge more ... Devices Surface PC Accessories Mobile Xbox PC Gaming HoloLens Hardware warranties Account & billing Account Microsoft Store & billing Resources Install Microsoft 365 Community forums Microsoft 365 Admins Small Business Portal Developer Education Report a support scam More Buy Microsoft 365 All Microsoft Global Microsoft 365 Teams Copilot Windows Surface Xbox Deals Small Business Support Software Software Windows Apps AI Outlook OneDrive Microsoft Teams OneNote Microsoft Edge Skype PCs & Devices PCs & Devices Computers Shop Xbox Accessories VR & mixed reality Certified Refurbished Trade-in f

## Multi URL Testing

In [76]:
# Retrieve all URLs, clean and append content, metadata and IDs

master_list = []

all_urls = get_all_urls()
for url in all_urls:
    html = fetch_html(url)
    if html:
        print(f"Got HTML dump!")

        cleaned_html = process_html(html, "html")
        document_id = str(uuid.uuid4())
        meta = {"source": url}
        
        item = {
            "document": cleaned_html,
            "metadata": meta,
            "id": document_id
        }
        master_list.append(item)

        print(f"item: {item}\n")

Got HTML dump!
Filtered top level tables count: 1

item: {'document': 'Introduction to tables - Microsoft Support Skip to main content Microsoft Support Support Support Home Microsoft 365 Office Products Microsoft 365 Outlook Microsoft Teams OneDrive OneNote Windows Microsoft Edge more ... Devices Surface PC Accessories Mobile Xbox PC Gaming HoloLens Hardware warranties Account & billing Account Microsoft Store & billing Resources Install Microsoft 365 Community forums Microsoft 365 Admins Small Business Portal Developer Education Report a support scam More Buy Microsoft 365 All Microsoft Global Microsoft 365 Teams Copilot Windows Surface Xbox Deals Small Business Support Software Software Windows Apps AI Outlook OneDrive Microsoft Teams OneNote Microsoft Edge Skype PCs & Devices PCs & Devices Computers Shop Xbox Accessories VR & mixed reality Certified Refurbished Trade-in for cash Entertainment Entertainment Xbox Game Pass Ultimate PC Game Pass Xbox games PC and Windows games Movies 

In [78]:
# Split documents into chunks

chunked_documents = []
for item in master_list:
    print(f"\nOriginal Item: {item}")
    chunks = chunk_text(item['document'], "cl100k_base", max_tokens=1024)
    for i, chunk in enumerate(chunks):
        new_id = item['id']
        new_id = f"{new_id}_{i+1}"

        new_meta = item['metadata'].copy()
        new_meta["ind"] = i

        new_item = {
            "document": chunk,
            "metadata": new_meta,
            "id": new_id
        }
        print(f"New Item: {new_item}")

        chunked_documents.append(new_item)


Original Item: {'document': 'Introduction to tables - Microsoft Support Skip to main content Microsoft Support Support Support Home Microsoft 365 Office Products Microsoft 365 Outlook Microsoft Teams OneDrive OneNote Windows Microsoft Edge more ... Devices Surface PC Accessories Mobile Xbox PC Gaming HoloLens Hardware warranties Account & billing Account Microsoft Store & billing Resources Install Microsoft 365 Community forums Microsoft 365 Admins Small Business Portal Developer Education Report a support scam More Buy Microsoft 365 All Microsoft Global Microsoft 365 Teams Copilot Windows Surface Xbox Deals Small Business Support Software Software Windows Apps AI Outlook OneDrive Microsoft Teams OneNote Microsoft Edge Skype PCs & Devices PCs & Devices Computers Shop Xbox Accessories VR & mixed reality Certified Refurbished Trade-in for cash Entertainment Entertainment Xbox Game Pass Ultimate PC Game Pass Xbox games PC and Windows games Movies & TV Business Business Microsoft Cloud Mi

In [79]:
# Split into 3 separate lists

documents_list = []
metadata_list = []
ids_list = []

for doc_info in chunked_documents:
    documents_list.append(doc_info['document'])
    metadata_list.append(doc_info['metadata'])
    ids_list.append(doc_info['id'])

print(f"Documents List: {documents_list}")
print(f"Metadata List: {metadata_list}")
print(f"IDs List: {ids_list}")

Documents List: ['Introduction to tables - Microsoft Support Skip to main content Microsoft Support Support Support Home Microsoft 365 Office Products Microsoft 365 Outlook Microsoft Teams OneDrive OneNote Windows Microsoft Edge more ... Devices Surface PC Accessories Mobile Xbox PC Gaming HoloLens Hardware warranties Account & billing Account Microsoft Store & billing Resources Install Microsoft 365 Community forums Microsoft 365 Admins Small Business Portal Developer Education Report a support scam More Buy Microsoft 365 All Microsoft Global Microsoft 365 Teams Copilot Windows Surface Xbox Deals Small Business Support Software Software Windows Apps AI Outlook OneDrive Microsoft Teams OneNote Microsoft Edge Skype PCs & Devices PCs & Devices Computers Shop Xbox Accessories VR & mixed reality Certified Refurbished Trade-in for cash Entertainment Entertainment Xbox Game Pass Ultimate PC Game Pass Xbox games PC and Windows games Movies & TV Business Business Microsoft Cloud Microsoft Secu

In [80]:
# Add data to ChromaDB

collection.add(
    documents=documents_list,
    metadatas=metadata_list,
    ids=ids_list
)

In [81]:
# Query ChromaDB

results = collection.query(
    query_texts=["table field"],
    n_results=3
)
results

{'ids': [['5b0c6be0-0d8f-428b-871c-5013a46b89d1_2',
   '3377e6b8-12b4-4a51-ac20-ada347bfb59b_2',
   '3a5d57d7-c98c-4fc1-9449-ec19fa71fcad_2']],
 'distances': [[1.079434871673584, 1.1747205257415771, 1.2338627576828003]],
 'metadatas': [[{'ind': 1,
    'source': 'https://support.microsoft.com/en-us/office/using-structured-references-with-excel-tables-f5ed2452-2337-4f71-bed3-c8ae6d2b276e'},
   {'ind': 1,
    'source': 'https://support.microsoft.com/en-us/office/introduction-to-tables-78ff21ea-2f76-4fb0-8af6-c318d1ee0ea7'},
   {'ind': 1,
    'source': 'https://support.microsoft.com/en-us/office/compare-two-tables-in-access-and-find-only-matching-data-16f301ac-40c1-43bc-80db-263f9a51eb4f'}]],
 'embeddings': None,
 'documents': [['the ribbon. Type the name you want in the Table Name box, and press Enter . In our example data, we used the name DeptSales . Use the following rules for table names: Use valid characters Always start a name with a letter, an underscore character ( _ ), or a backs

In [84]:
payload = {
    'messages': [
        {
            'role': 'system', 
            'content': f"You are a helpful assistant. You will be provided context to the question the user is asking. Without any prior knowledge assumptions ONLY answer their qustion."
        },
        {
            'role': 'user',
            'content': f"What type of field would be created if i enter €1.23?\n{results}"
        }
    ],
    'model': "gpt-3.5-turbo",
    'max_tokens': 128,
    "temperature": 0.0,
    'seed': 48
}
input_tokens, output_tokens, response = make_openai_call(payload)
print(f"input tokens: {input_tokens}")
print(f"output tokens: {output_tokens}")
print(f"response: {response}")

input tokens: 3448
output tokens: 19
response: A field with a data type of Currency would be created if you enter €1.23.
